In [27]:
from langchain.chat_models import AzureChatOpenAI

# Use azure api

BASE_URL = "https://conslearn.openai.azure.com"
API_KEY = "fcb9227a52b640d387bfe43097ebaa25"
DEPLOYMENT_NAME = 'gpt35'

llm = AzureChatOpenAI(
    openai_api_base=BASE_URL,
    openai_api_version="2023-09-01-preview",
    deployment_name=DEPLOYMENT_NAME,
    openai_api_key=API_KEY,
    openai_api_type="azure",
    temperature=0,
    verbose=True
)

In [42]:
# test for azure
from langchain.callbacks import get_openai_callback
from langchain.schema import HumanMessage

with get_openai_callback() as cb:
    llm(
        [
            HumanMessage(
                content="Translate this sentence from English to French. I love programming."
            )
        ]
    )
    print(
        f"Total Cost (USD): ${format(cb.total_cost, '.6f')}"
    )  # without specifying the model version, flat-rate 0.002 USD per 1k input and output tokens is used

[llm/start] [1:llm:AzureChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Translate this sentence from English to French. I love programming."
  ]
}
[llm/end] [1:llm:AzureChatOpenAI] [1.57s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "J'adore la programmation.",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "J'adore la programmation.",
            "additional_kwargs": {}
          }
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 19,
      "completion_tokens": 8,
      "total_tokens": 27
    },
    "model_name": "gpt-35-turbo"
  },
  "run": null
}
Total Cost (USD): $0.000044


In [31]:
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

tools = [get_word_length]

In [29]:
from langchain.globals import set_debug, set_verbose

set_debug(False)

In [32]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are very powerful assistant, but bad at calculating lengths of words."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [33]:
from langchain.tools.render import format_tool_to_openai_function
llm_with_tools = llm.bind(
    functions=[format_tool_to_openai_function(t) for t in tools]
)

In [34]:
# LCEL stands for LangChain Expression Language. It is a language used in the LangChain framework 
# for composing and executing different components such as prompt templates, language models, and 
# output parsers. LCEL provides a way to easily combine these components using a pipeline-like syntax.
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps'])
} | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()

In [35]:
st = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps'])
} | prompt |llm_with_tools

In [43]:
set_debug(True)
set_verbose(True)
agent.invoke({"input": "When is now?",
              "intermediate_steps": []})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "When is now?",
  "intermediate_steps": []
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel] Entering Chain run with input:
{
  "input": "When is now?",
  "intermediate_steps": []
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:<lambda>] Entering Chain run with input:
{
  "input": "When is now?",
  "intermediate_steps": []
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:<lambda>] s] Exiting Chain run with output:
{
  "output": "When is now?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 4:chain:<lambda>] Entering Chain run with input:
{
  "input": "When is now?",
  "intermediate_steps": []
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 4:chain:<lambda>] s] Exiting Chain run with output:
{
  "output": []
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel] s] Exit

AgentFinish(return_values={'output': 'Now is the present moment. It is the current time or the time at which something is happening.'}, log='Now is the present moment. It is the current time or the time at which something is happening.')

In [40]:
agent.invoke({
    "input": "how many letters in the word educa?",
    "intermediate_steps": []
})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "how many letters in the word educa?",
  "intermediate_steps": []
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel] Entering Chain run with input:
{
  "input": "how many letters in the word educa?",
  "intermediate_steps": []
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:<lambda>] Entering Chain run with input:
{
  "input": "how many letters in the word educa?",
  "intermediate_steps": []
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:<lambda>] s] Exiting Chain run with output:
{
  "output": "how many letters in the word educa?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 4:chain:<lambda>] Entering Chain run with input:
{
  "input": "how many letters in the word educa?",
  "intermediate_steps": []
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 4:chain:<lambda>] s] Exiting 

AgentActionMessageLog(tool='get_word_length', tool_input={'word': 'educa'}, log="\nInvoking: `get_word_length` with `{'word': 'educa'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_word_length', 'arguments': '{\n  "word": "educa"\n}'}})])

In [41]:
from langchain.schema.agent import AgentFinish
intermediate_steps = []
while True:
    output = agent.invoke({
        "input": "how many letters in the word educa?",
        "intermediate_steps": intermediate_steps
    })
    if isinstance(output, AgentFinish):
        final_result = output.return_values["output"]
        break
    else:
        print(output.tool, output.tool_input)
        tool = {
            "get_word_length": get_word_length
        }[output.tool]
        observation = tool.run(output.tool_input)
        intermediate_steps.append((output, observation))
print(final_result)

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "how many letters in the word educa?",
  "intermediate_steps": []
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel] Entering Chain run with input:
{
  "input": "how many letters in the word educa?",
  "intermediate_steps": []
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:<lambda>] Entering Chain run with input:
{
  "input": "how many letters in the word educa?",
  "intermediate_steps": []
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:<lambda>] s] Exiting Chain run with output:
{
  "output": "how many letters in the word educa?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 4:chain:<lambda>] Entering Chain run with input:
{
  "input": "how many letters in the word educa?",
  "intermediate_steps": []
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 4:chain:<lambda>] s] Exiting 

In [11]:
for step in agent["agent_scratchpad"]:
    print(f"Input: {step['input']}")
    print(f"Output: {step['output']}")
    print()

TypeError: 'RunnableSequence' object is not subscriptable

In [12]:
input_data = agent("input")
scratchpad_data = agent("agent_scratchpad")

TypeError: 'RunnableSequence' object is not callable